In [19]:
import datetime
import os


import difflib
import dotenv
import googlemaps
import pandas as pd


# Long Term Care Data from Covid Tracking Project

A subset of Cook County longterm care data from a single date was taken to examine the shape and fields of available data. Two things jumped out: the facility_cms_id was missing for several entries, and no address data was given that could easily link facilities to their CMS profile. 

In [20]:
df_cases = pd.read_csv("facilities_il.csv")
df_cases['date']  = pd.to_datetime(df_cases['date'], format="%Y%m%d")
df_cases = df_cases.dropna(1, how="all")
df_cases["facility_name"] = df_cases["facility_name"].str.upper()
df_cases = df_cases[df_cases["county"]=="Cook"]
df_cases = df_cases[df_cases["date"] == "2020-12-03"]
cases_names = df_cases.set_index("facility_name").sort_index()[0:50]
cases_names

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (6,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,state,county,ctp_facility_category,state_fed_regulated,facility_cms_id,outbreak_status,resident_staff_positives,resident_staff_deaths
facility_name,,,,,,,,,
ABINGTON OF GLENVIEW,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145683,NaN,8,1
ADDOLORATA VILLA,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145724,NaN,112,20
ALBANY CARE,2020-12-03,IL,Cook,Uncategorized LTC,Federal,NaN,NaN,39,0
ALDEN,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145998,NaN,37,2
ALDEN ESTATES DES PLAINES,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145998,NaN,26,4
ALDEN ESTATES OF BARRINGTON,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145557,NaN,36,0
ALDEN ESTATES OF EVANSTON,2020-12-03,IL,Cook,NaN,NaN,145907,NaN,18,2
ALDEN ESTATES OF NORTHMOOR,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145888,NaN,98,19
ALDEN ESTATES OF ORLAND PARK,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145963,NaN,45,13


#### Centers for Medicare and Medicade Services (CMS) Data
After noticing that facility_cms_id was incomplete in the Covid Tracking Project dataset, I wanted to see what type of information was present in the CMS data. We can see that there is a plethora of fields that might affect covid care, including CMS rankings of the facilities, historical fines and complaints, and ownership type (i.e. for profit vs. nonprofit). 

In [21]:
df_facility = pd.read_csv("provider_info.csv", encoding = "ISO-8859-1")
df_facility = df_facility[df_facility["Provider State"] == "IL"]
df_facility = df_facility[df_facility["Provider County Name"] == "Cook"]
df_facility["Provider Name"] = df_facility["Provider Name"].str.replace(", THE","")
facility_names = df_facility[["Provider Name", "Federal Provider Number"]].set_index("Provider Name").sort_index()
df_facility

,Federal Provider Number,Provider Name,Provider Address,Provider City,Provider State,Provider Zip Code,Provider Phone Number,Provider SSA County Code,Provider County Name,Ownership Type,...,Rating Cycle 3 Total Health Score,Total Weighted Health Survey Score,Number of Facility Reported Incidents,Number of Substantiated Complaints,Number of Fines,Total Amount of Fines in Dollars,Number of Payment Denials,Total Number of Penalties,Location,Processing Date
2964,145011,GROVE OF EVANSTON L & R,500 ASBURY STREET,EVANSTON,IL,60202,8473163320,141,Cook,For profit - Limited Liability company,...,28,33.333,1,4,1,13500,0,1,"500 ASBURY STREET,EVANSTON,IL,60202",2020-11-01
2969,145026,WESTMINSTER PLACE,3200 GRANT STREET,EVANSTON,IL,60201,8474924800,141,Cook,Non profit - Other,...,76,44.000,0,0,0,0,0,0,"3200 GRANT STREET,EVANSTON,IL,60201",2020-11-01
2980,145070,GROVE OF BERWYN,3601 SOUTH HARLEM AVENUE,BERWYN,IL,60402,7087494160,141,Cook,For profit - Limited Liability company,...,40,128.667,0,22,1,6000,0,1,"3601 SOUTH HARLEM AVENUE,BERWYN,IL,60402",2020-11-01
2981,145087,MANORCARE OF OAK LAWN WEST,6300 WEST 95TH STREET,OAK LAWN,IL,60453,7085998800,141,Cook,Non profit - Other,...,24,56.667,0,2,0,0,0,0,"6300 WEST 95TH STREET,OAK LAWN,IL,60453",2020-11-01
2985,145122,DOBSON PLAZA,120 DODGE AVENUE,EVANSTON,IL,60202,8478697744,141,Cook,For profit - Corporation,...,12,4.000,0,0,0,0,0,0,"120 DODGE AVENUE,EVANSTON,IL,60202",2020-11-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3646,146189,LITTLE SISTERS OF THE POOR OF PALATINE,80 WEST NORTHWEST HIGHWAY,PALATINE,IL,60067,8473585700,141,Cook,Non profit - Corporation,...,.,24.800,0,0,0,0,0,0,"80 WEST NORTHWEST HIGHWAY,PALATINE,IL,60067",2020-11-01
3648,146191,MADO HEALTHCARE - UPTOWN,4621 NORTH RACINE AVENUE,CHICAGO,IL,60640,7737842300,141,Cook,For profit - Corporation,...,.,86.400,0,0,0,0,0,0,"4621 NORTH RACINE AVENUE,CHICAGO,IL,60640",2020-11-01
3650,14A057,ALL AMERICAN NURSING HOME,5448 NORTH BROADWAY STREET,CHICAGO,IL,60640,7733342224,141,Cook,For profit - Corporation,...,68,68.000,0,8,0,0,0,0,"5448 NORTH BROADWAY STREET,CHICAGO,IL,60640",2020-11-01
3656,14E169,WINSTON MANOR CNV & NURSING,2155 WEST PIERCE,CHICAGO,IL,60622,7732522066,141,Cook,For profit - Corporation,...,20,29.333,0,3,0,0,0,0,"2155 WEST PIERCE,CHICAGO,IL,60622",2020-11-01


### Attempt at Fuzzy Matching

I resolved to match the Covid Tracking Project (CTP) data to the CMS data. I contemplated the idea that the CTP field of "facility_name" might be a fuzzy match with the CMS field of "Provider Name" and found a Python library for [determine close word matches](https://www.kite.com/python/docs/difflib.get_close_matches). However, upon examine the joined data, I determined there were too unmatched (a None in the "matched_name" field) and incorrectly matched facilities to make this approach worthwhile.   

In [18]:
def close_match(x, index):
    match = difflib.get_close_matches(x, index, cutoff=.7)
    if len(match) > 0:
        return match[0]
    else:
        return None
cases_names["matched_name"] = cases_names.index.map(lambda x: close_match(x, facility_names.index))
cases_names

,date,state,county,ctp_facility_category,state_fed_regulated,facility_cms_id,outbreak_status,resident_staff_positives,resident_staff_deaths,matched_name
facility_name,,,,,,,,,,
ABINGTON OF GLENVIEW,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145683,NaN,8,1,ABINGTON OF GLENVIEW NURSING
ADDOLORATA VILLA,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145724,NaN,112,20,ADDOLORATA VILLA
ALBANY CARE,2020-12-03,IL,Cook,Uncategorized LTC,Federal,NaN,NaN,39,0,None
ALDEN,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145998,NaN,37,2,None
ALDEN ESTATES DES PLAINES,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145998,NaN,26,4,None
ALDEN ESTATES OF BARRINGTON,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145557,NaN,36,0,ALDEN ESTATES OF BARRINGTON
ALDEN ESTATES OF EVANSTON,2020-12-03,IL,Cook,NaN,NaN,145907,NaN,18,2,ALDEN ESTATES OF EVANSTON
ALDEN ESTATES OF NORTHMOOR,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145888,NaN,98,19,ALDEN ESTATES OF NORTHMOOR
ALDEN ESTATES OF ORLAND PARK,2020-12-03,IL,Cook,Uncategorized LTC,Federal,145963,NaN,45,13,ALDEN ESTATES OF ORLAND PARK


### Potential Geolocation
Based on another project I had been working on at the time, I resolved that it would be possible to augment the CTP data with addresses by looking up the facility name and county with the Google Maps API, which would return at lat & long as well as a street address.
The return rate of addresses was better than the fuzzy match rate above, so I built this geolocation augmentation into my data pipeline through a Python script.

In [22]:
from dotenv import load_dotenv
load_dotenv()
gmaps = googlemaps.Client(os.getenv("GOOGLE_MAPS_API_KEY"))

In [23]:
coordinates = []
addresses = []
types = []
for fac_name in cases_names.index:
    lookup = f"{fac_name}, Cook County, IL"
    result = gmaps.geocode(lookup)[0]
    address = result["formatted_address"]
    coord = result["geometry"]["location"]
    types = result["types"]
    health = "health" in types
    if health:
        coordinates.append(coord)
        addresses.append(address)
    else:
        coordinates.append(None)
        addresses.append(None)
coordinates

[{'lat': 42.07238299999999, 'lng': -87.854056},
 {'lat': 42.1475952, 'lng': -87.94060549999999},
 {'lat': 42.0342403, 'lng': -87.68507799999999},
 None,
 {'lat': 42.0538642, 'lng': -87.89333909999999},
 {'lat': 42.1329079, 'lng': -88.13764189999999},
 {'lat': 42.0633128, 'lng': -87.73018599999999},
 {'lat': 41.9878101, 'lng': -87.7904245},
 {'lat': 41.591145, 'lng': -87.856038},
 {'lat': 42.0628872, 'lng': -87.74305319999999},
 {'lat': 42.0628872, 'lng': -87.74305319999999},
 {'lat': 41.9694017, 'lng': -87.65082629999999},
 {'lat': 41.936854, 'lng': -87.642162},
 {'lat': 42.0121369, 'lng': -87.7544211},
 {'lat': 42.0121369, 'lng': -87.7544211},
 {'lat': 42.05163, 'lng': -88.141857},
 {'lat': 42.05163, 'lng': -88.141857},
 {'lat': 41.7685822, 'lng': -87.63176299999999},
 {'lat': 41.833187, 'lng': -87.77694799999999},
 {'lat': 41.833187, 'lng': -87.77694799999999},
 {'lat': 42.0179687, 'lng': -87.6659878},
 {'lat': 41.7687078, 'lng': -87.6304378},
 {'lat': 41.7687078, 'lng': -87.6304378}